In [ ]:
## NAME: Laila Ali Yassin Cruz
# Assignment 1

# Importing the data #
import pandas as pd
path = 'https://raw.githubusercontent.com/cinnData/MLearning/main/Data/'
df = pd.read_csv(path + 'king.csv', index_col=0)

# Exploring the data #
df.info()
df.index.duplicated().sum()
df.duplicated().sum()
duplicates = df.index[df.index.duplicated()]
df.loc[duplicates].head()
df['price'] = df['price']/1000

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21613 entries, 7129300520 to 1523300157
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           21613 non-null  object 
 1   zipcode        21613 non-null  int64  
 2   lat            21613 non-null  float64
 3   long           21613 non-null  float64
 4   bedrooms       21613 non-null  int64  
 5   bathrooms      21613 non-null  float64
 6   sqft_above     21613 non-null  int64  
 7   sqft_basement  21613 non-null  int64  
 8   sqft_lot       21613 non-null  int64  
 9   floors         21613 non-null  float64
 10  waterfront     21613 non-null  int64  
 11  condition      21613 non-null  int64  
 12  yr_built       21613 non-null  int64  
 13  yr_renovated   21613 non-null  int64  
 14  price          21613 non-null  int64  
dtypes: float64(4), int64(10), object(1)
memory usage: 2.6+ MB


## Q1. The role of longitude and latitude in the prediction of real estate prices is unclear. Do they really contribute to get better predictions in the first model of this example? If we keep them in the second model, do we get a better model?

In [ ]:
# We obtain the first model's prediction strength including latitude and longitute

# Linear regression equation #
y = df.iloc[:, -1]
X_model1 = df.iloc[:, 2:-1]
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_model1, y)
y_pred_model1 = reg.predict(X_model1)
reg.score(X_model1, y).round(3)

0.646

In [ ]:
# We obtain the first model's prediction strength removing latitude and longitute

# Linear regression equation
y = df.iloc[:, -1]
X_2 = df.iloc[:, 4:-1] # we start the dataframe from the fourth column to remove
reg.fit(X_2, y)
y_pred_2 = reg.predict(X_2)
reg.score(X_2, y).round(3)

0.587

In [ ]:
#We now do the same comparison with the second model, using zipcodes as dummies

# Q4. Dummies for the zipcodes #
X_model2_1 = df.iloc[:, 2:-1]
X_model2_2 = pd.get_dummies(df['zipcode'])
X_model2_2.head()
X_model2 = pd.concat([X_model2_1, X_model2_2], axis=1)
X_model2.shape
X_model2 = X_model2.values
reg.fit(X_model2, y)
y_pred_model2 = reg.predict(X_model2)
reg.score(X_model2, y).round(3)

0.786

In [ ]:
#We now do the same comparison with the second model, but removing latitude and longitude

# Q4. Dummies for the zipcodes #
X1 = df.iloc[:, 4:-1]  #we start the dataframe from the fourth column to remove lat and long
X2 = pd.get_dummies(df['zipcode'])
X2.head()
X = pd.concat([X1, X2], axis=1)
X.shape
X = X.values
reg.fit(X, y)
y_pred = reg.predict(X)
reg.score(X, y).round(3)

0.785

## **Answer Q1:**
We're seeing that while for the first model the prediction score deteriorates from 64.6% to 58.7% by removing latitude and longitude,once we start using the zipcodes as dummies the impact of removing latitude and longitude in the model just makes the model lose prediction in 0.1%, going from 78.6% to 78.5%). This may be because the effect of lat and longitude is already being captured in a great part by zipcode.


## Q2. Evaluate in dollar terms the predictive performance of the two models presented in this example. For instance, you can use the mean (or median) absolute error. Can you make a statement like "the value of x% of the houses can be predicted with an error below y thousand dollars"?

In [ ]:
# We add the predicted prices in our dataframe
df1 =df
df1['y_pred_model1'] = y_pred_model1
df1['y_pred_model2'] = y_pred_model2

In [ ]:
# We obtain the model's errors
df1['error_model1'] = abs(df1['y_pred_model1']- df1['price'])
df1['error_model2'] = abs(df1['y_pred_model2']- df1['price'])

## **Answer Q2: Model 1**
We can see that in error terms, the value of 50% of the houses will be predicted with an error below 97,000 dollars

In [ ]:
df1['error_model1'].describe()

count    21613.000000
mean       139.328120
std        168.191617
min          0.018830
25%         45.393321
50%         97.696843
75%        177.025142
max       3922.773169
Name: error_model1, dtype: float64

## **Answer Q2: Model 2**
We can see that in error terms, the value of 50% of the houses will be predicted with an error below 70,000 dollars

In [ ]:
df1['error_model2'].describe()

count    21613.000000
mean       103.057215
std        135.093873
min          0.006001
25%         32.893924
50%         70.611755
75%        126.538218
max       4207.544880
Name: error_model2, dtype: float64

## Q3. Is it better to use the percentage error in the above assessment?

In [ ]:
# We obtain the model's errors as a percentage of of the real price
df1['error_model1_perc'] = df1['error_model1']/ df1['price']

df1['error_model2_perc'] = df1['error_model2']/ df1['price']

## **Answer Q3: Model 1**
We can see that in percentage terms, the value of 50% of the houses will be predicted with an error below 20% of their actual value

In [ ]:
df1['error_model1_perc'].describe()

count    21613.000000
mean         0.279175
std          0.251885
min          0.000051
25%          0.100224
50%          0.216834
75%          0.380352
max          3.254671
Name: error_model1_perc, dtype: float64

## **Answer Q3: Model 2**
We can see that in percentage terms, the value of 50% of the houses will be predicted with an error below 15% of their actual value

In [ ]:
df1['error_model2_perc'].describe()

count    21613.000000
mean         0.208424
std          0.199224
min          0.000012
25%          0.071493
50%          0.154808
75%          0.284807
max          3.166680
Name: error_model2_perc, dtype: float64

In this instance, since the value of a house is very different from one to the other, seeing the errors in percentage points helps us see the magnitudes of the errors in relative terms to the price of the house

## Q4. Can the strong correlation be an artifact created by the extreme values? Trim the data set, dropping the houses beyond a certain threshold of price and/or size. Do you get a better model?

In [ ]:
#To remove outliers first we understand the distribution of our y variable (price)

df['price'].describe()

count    21613.000000
mean       540.088142
std        367.127196
min         75.000000
25%        321.950000
50%        450.000000
75%        645.000000
max       7700.000000
Name: price, dtype: float64

In [ ]:
import numpy as np
from scipy import stats
# We obtain the z-score for our chosen variable, in this case price
z = np.abs(stats.zscore(df['price']))

In [ ]:
threshold = 3
outliers = df[z > threshold]

# We print the outliers (we can see we're removing 406 rows out of 21613, approximately 1.8% of the date
print(outliers)

                       date  zipcode      lat     long  bedrooms  bathrooms  \
id                                                                            
2524049179  20140826T000000    98040  47.5316 -122.233         3       2.75   
7855801670  20150401T000000    98006  47.5620 -122.162         4       3.25   
2025069065  20140929T000000    98074  47.6338 -122.072         4       2.50   
7960900060  20150504T000000    98004  47.6312 -122.223         4       3.25   
7424700045  20150513T000000    98122  47.6166 -122.287         5       3.00   
...                     ...      ...      ...      ...       ...        ...   
3262300818  20150227T000000    98039  47.6351 -122.236         4       3.75   
8964800330  20150407T000000    98004  47.6200 -122.207         4       3.75   
715010530   20150113T000000    98006  47.5382 -122.111         5       3.50   
524059330   20150130T000000    98004  47.5990 -122.197         4       3.50   
9253900271  20150107T000000    98008  47.5943 -122.1

In [ ]:
df_clean = df[z < threshold]

In [ ]:
# We obtain the first model's prediction strength, without outliers

# Linear regression equation #
y = df_clean.iloc[:, -1]
X_model1 = df_clean.iloc[:, 2:-1]
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_model1, y)
y_pred_model1 = reg.predict(X_model1)
reg.score(X_model1, y).round(3)

0.624

In [ ]:
#We now do the same comparison with the second model, using zipcodes as dummies

# Q4. Dummies for the zipcodes #
X_model2_1 = df_clean.iloc[:, 2:-1]
X_model2_2 = pd.get_dummies(df_clean['zipcode'])
X_model2_2.head()
X_model2 = pd.concat([X_model2_1, X_model2_2], axis=1)
X_model2.shape
X_model2 = X_model2.values
reg.fit(X_model2, y)
y_pred_model2 = reg.predict(X_model2)
reg.score(X_model2, y).round(3)

0.799

## **Answer Q4: Dropping price outliers**
We can see that dropping price outliers (removing values that are more than 3 stds beyond the mean), the first model drops in prediction power while the second model improves in almost 1.5%

In [ ]:
# We repeat what we did but now choosing sqft_above as proxy for size
#We obtain the z-score for our chosen variable, in this case sqft_above
z_size = np.abs(stats.zscore(df['sqft_above']))

In [ ]:
threshold = 3
outliers_size = df[z_size > threshold]

# We print the outliers (we can see we're removing 254 rows out of 21613, approximately 1.2% of the date
print(outliers_size)

                       date  zipcode      lat     long  bedrooms  bathrooms  \
id                                                                            
7960900060  20150504T000000    98004  47.6312 -122.223         4       3.25   
4054500390  20141007T000000    98077  47.7285 -122.042         4       4.75   
8678500060  20140710T000000    98024  47.5954 -121.950         5       4.25   
2625069070  20150410T000000    98074  47.6179 -122.005         4       3.25   
7322910030  20140721T000000    98053  47.6554 -122.018         5       3.50   
...                     ...      ...      ...      ...       ...        ...   
2625069038  20141124T000000    98074  47.6258 -122.005         4       3.50   
324069112   20140617T000000    98075  47.5914 -122.027         4       4.00   
1561750040  20141224T000000    98074  47.6018 -122.060         5       4.50   
6664500090  20150115T000000    98059  47.4832 -122.145         5       4.00   
249000205   20141015T000000    98004  47.6321 -122.2

In [ ]:
df_clean_size = df[z_size < threshold]

In [ ]:
# We obtain the first model's prediction strength, without outliers

# Linear regression equation #
y = df_clean_size.iloc[:, -1]
X_model1 = df_clean_size.iloc[:, 2:-1]
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_model1, y)
y_pred_model1 = reg.predict(X_model1)
reg.score(X_model1, y).round(3)

0.614

In [ ]:
#We now do the same comparison with the second model, using zipcodes as dummies

# Q4. Dummies for the zipcodes #
X_model2_1 = df_clean_size.iloc[:, 2:-1]
X_model2_2 = pd.get_dummies(df_clean_size['zipcode'])
X_model2_2.head()
X_model2 = pd.concat([X_model2_1, X_model2_2], axis=1)
X_model2.shape
X_model2 = X_model2.values
reg.fit(X_model2, y)
y_pred_model2 = reg.predict(X_model2)
reg.score(X_model2, y).round(3)

0.776

## **Answer Q4: Dropping size outliers**
We can see that by dropping size outliers (removing values that are more than 3 stds beyond the mean), the first model drops in prediction power and the second model also drops in prediction power.
This could be because size is an important variable to keep to avoid overfitting the model to the standard values

## Q5. The distribution of the price is quite skewed, which is a fact of life in real state. The extreme values in the right tail of the distribution can exert an undesired influence on the regression coefficients. Develop and evaluate a model for predicting the price that is based on a linear regression equation which has the logarithm of the price on the left side.

In [ ]:
# We obtain the first model's prediction strength

# Linear regression equation #
df2 = df
df2['price'] = np.log(df2['price']) #we obtain log of price
y_log = df2['price']
X_model1 = df2.iloc[:, 2:-1]
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_model1, y)
y_pred_model1 = reg.predict(X_model1)
reg.score(X_model1, y).round(3)

0.697

In [ ]:
#We now do the same comparison with the second model, using zipcodes as dummies

# Q4. Dummies for the zipcodes #
X_model2_1 = df2.iloc[:, 2:-1]
X_model2_2 = pd.get_dummies(df2['zipcode'])
X_model2_2.head()
X_model2 = pd.concat([X_model2_1, X_model2_2], axis=1)
X_model2.shape
X_model2 = X_model2.values
reg.fit(X_model2, y_log)
y_pred_model2 = reg.predict(X_model2)
reg.score(X_model2, y_log).round(3)

0.837

## **Answer Q5:**
We can see that by transforming price through its logarithm, both models increase in prediction power. Model 1 goes from 64.6% to 69.7%, while model 2 goes from 78.7% to 83.7%. With these improvements, we can conclude we should consider using the logarithm of price instead.